In [1]:
import pandas as pd
import numpy as np

In [44]:
# Raw data
size = 'large'
articles_path = f'./files/parquet/ebnerd_{size}/articles.parquet'
articles_df = pd.read_parquet(articles_path)
print(articles_df.shape)

size = 'demo'
history_val_path = f'./files/parquet/ebnerd_{size}/validation/history.parquet'
history_val_df = pd.read_parquet(history_val_path)
print(history_val_df.shape)

# Read interaction matrix and pre-process
size = 'demo'
interaction_matrix_path = f'./files/csv/interaction_matrix_{size}.csv'
interaction_matrix_df = pd.read_csv(interaction_matrix_path)
print(interaction_matrix_df.shape)
interaction_matrix_df.set_index('user_id', inplace=True)
interaction_matrix_df = interaction_matrix_df.replace({np.nan: 0})

# Read cosine similarity matrix pickle file
cosine_similarity_df_file_path = './files/pickle/cosine_similarity_matrix_50K_x_50K.pkl'
cosine_similarity_df = pd.read_pickle(cosine_similarity_df_file_path)

# Read user matrix and article matrix pickle file
user_matrix_df_file_path = './files/pickle/user_matrix.pkl'
article_matrix_df_file_path = './files/pickle/article_matrix_df.pkl'

user_matrix_df = pd.read_pickle(user_matrix_df_file_path)
article_matrix_df = pd.read_pickle(article_matrix_df_file_path)
print(user_matrix_df.shape)
print(article_matrix_df.shape)



(125541, 21)
(1562, 5)
(1590, 4248)
(1590, 5)
(4247, 5)


In [45]:
interaction_matrix_df.head()

,9251369,9730301,9733713,9737535,9738292,9738334,9740021,9740161,9740174,9740356,...,9700074,9737345,6404190,9448400,9728595,9726404,8166777,9582969,9627627,9674356
user_id,,,,,,,,,,,,,,,,,,,,,
11313,199.5,210.5,4.0,549.0,66.0,52.0,91.0,25.5,437.0,38.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13538,0.0,12.5,7.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15430,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19181,0.0,0.0,0.0,0.0,15.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
cosine_similarity_df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,49990,49991,49992,49993,49994,49995,49996,49997,49998,49999
article_id,,,,,,,,,,,,,,,,,,,,,
3000022,1.000000,0.824758,0.780845,0.827969,0.796304,0.801253,0.822717,0.841739,0.805518,0.672523,...,0.826866,0.774553,0.753119,0.766686,0.762471,0.846480,0.668666,0.767384,0.766419,0.770445
3000063,0.824758,1.000000,0.846127,0.881168,0.890629,0.858967,0.907076,0.847015,0.897508,0.774397,...,0.896323,0.842242,0.843697,0.852669,0.868081,0.903028,0.780654,0.820704,0.842309,0.901539


In [47]:
history_val_df.head()

,user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
0,750497,"[2023-05-04T09:42:39.000000, 2023-05-04T09:43:...","[100.0, 65.0, 100.0, 100.0, 100.0, 100.0, 100....","[9749224, 9749156, 9749224, 9748948, 9748980, ...","[49.0, 5.0, 7.0, 151.0, 214.0, 199.0, 22.0, 64..."
1,22779,"[2023-05-04T07:53:42.000000, 2023-05-04T15:59:...","[52.0, 39.0, 62.0, 38.0, 74.0, 19.0, 30.0, 56....","[9749025, 9750090, 9750015, 9750161, 9745750, ...","[4.0, 16.0, 2.0, 9.0, 40.0, 7.0, 9.0, 8.0, 18...."
2,373598,"[2023-05-04T07:51:58.000000, 2023-05-04T09:59:...","[nan, nan, nan, 59.0, 33.0, 75.0, nan, nan, 76...","[9514481, 9514481, 9111040, 9750389, 9750307, ...","[0.0, 0.0, 0.0, 3.0, 9.0, 117.0, 39.0, 0.0, 8...."
3,383378,"[2023-05-04T07:27:57.000000, 2023-05-04T07:29:...","[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100...","[9747490, 9749036, 9749025, 9748792, 9748592, ...","[85.0, 18.0, 133.0, 191.0, 331.0, 56.0, 43.0, ..."
4,411733,"[2023-05-04T17:09:09.000000, 2023-05-04T17:09:...","[20.0, 14.0, 61.0, 55.0, 21.0, 81.0, 100.0, 10...","[9750081, 9750111, 9750039, 9749948, 9749729, ...","[2.0, 4.0, 6.0, 9.0, 1.0, 30.0, 37.0, 5.0, 3.0..."


In [48]:
list(history_val_df[['article_id_fixed']].loc[0,:][0])

/var/folders/ld/c07vs56s18bc0h2krlvl0myr0000gn/T/ipykernel_3473/3053616938.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  list(history_val_df[['article_id_fixed']].loc[0,:][0])


[9749224,
 9749156,
 9749224,
 9748948,
 9748980,
 9748792,
 9741033,
 9748792,
 9748792,
 9748916,
 9749076,
 9749948,
 9749947,
 9749635,
 9749444,
 9748798,
 9749397,
 9748798,
 9749916,
 9745750,
 9749668,
 9745750,
 9750296,
 9745750,
 9749916,
 9750318,
 9749886,
 9749240,
 9747411,
 9749886,
 9749886,
 9751290,
 9751220,
 9751385,
 9751020,
 9751385,
 9751135,
 9751385,
 9751385,
 9750789,
 9638857,
 9751020,
 9750687,
 9749759,
 9750687,
 9750904,
 9750687,
 9751385,
 9751411,
 9749184,
 9751411,
 9749634,
 9750995,
 9751452,
 9751411,
 9751897,
 9751850,
 9751895,
 9751866,
 9744479,
 9751866,
 9751895,
 9751895,
 9747369,
 9751895,
 9751764,
 9751670,
 9744480,
 9744479,
 9744480,
 9751688,
 9751524,
 9751932,
 9751932,
 9752155,
 9752155,
 9751975,
 9751877,
 9751975,
 9751557,
 9751771,
 9751557,
 9735795,
 9751557,
 9751778,
 9751557,
 9751975,
 9752155,
 9725978,
 9752155,
 9749182,
 9752243,
 9749938,
 9752340,
 9752323,
 9749966,
 9752155,
 7787495,
 9752155,
 9751646,


In [76]:
# Function to get recommendations based on content similarity
def get_recommendations(article_id, cosine_similarity_df=cosine_similarity_df):
    idx = articles_df[articles_df['article_id'] == article_id].index[0]
    sim_scores = list(enumerate(cosine_similarity_df[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Top 10 recommendations
    article_indices = [i[0] for i in sim_scores]
    return (articles_df['article_id'].iloc[article_indices], articles_df[articles_df['article_id'] == article_id])

# Example usage
print(get_recommendations(3001353))

(38200    5969633
16615    4329966
11393    4186579
18089    4374698
27497    4776283
11394    4186589
5352     4045124
18984    4399314
913      3182187
22148    4509154
Name: article_id, dtype: int32,    article_id                         title  \
7     3001353  Natascha var ikke den første   

                                            subtitle  last_modified_time  \
7  Politiet frygter nu, at Nataschas bortfører ha... 2023-06-29 06:20:33   

   premium                                               body  \
7    False  Sagen om den østriske Natascha og hendes bortf...   

       published_time  image_ids     article_type  \
7 2006-08-31 08:06:45  [3150850]  article_default   

                                                url  ... entity_groups  \
7  https://ekstrabladet.dk/krimi/article3001353.ece  ...            []   

                                      topics category  subcategory  \
7  [Kriminalitet, Personfarlig kriminalitet]      140           []   

  category_str total_

In [52]:
750497 in interaction_matrix_df.index 

True

In [54]:
interaction_matrix_df.loc[750497, :]

9251369    0.0
9730301    0.0
9733713    0.0
9737535    0.0
9738292    0.0
          ... 
9726404    0.0
8166777    0.0
9582969    0.0
9627627    0.0
9674356    0.0
Name: 750497, Length: 4247, dtype: float64

In [55]:
article_id = 3001353
user_id = 750497

In [57]:
interaction_matrix_df.index.get_loc(user_id)

474

In [58]:
interaction_matrix_df

,9251369,9730301,9733713,9737535,9738292,9738334,9740021,9740161,9740174,9740356,...,9700074,9737345,6404190,9448400,9728595,9726404,8166777,9582969,9627627,9674356
user_id,,,,,,,,,,,,,,,,,,,,,
11313,199.5,210.5,4.0,549.0,66.0,52.0,91.0,25.5,437.0,38.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13538,0.0,12.5,7.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15430,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19181,0.0,0.0,0.0,0.0,15.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2581162,0.0,17.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,63.0,0.0,0.0,0.0,0.0
2583035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2583477,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0,0.0,201.0


In [60]:
user_pref = user_matrix_df.loc[user_id, :]

In [61]:
user_pref

0    0.004148
1    4.442545
2    0.502907
3    0.797032
4    1.204397
Name: 750497, dtype: float64

In [64]:
cf_scores = user_pref.dot(article_matrix_df.values.T)

In [65]:
cf_scores

array([ 0.37752355, 28.0474402 ,  9.23650773, ...,  0.1090541 ,
        0.        ,  0.27746675])

In [66]:
cf_recs = interaction_matrix_df.columns[cf_scores.argsort()[::-1]]

In [67]:
cf_recs

Index(['9765410', '9725978', '9766042', '9742824', '9737243', '9762377',
       '9764361', '9748576', '9755964', '9759355',
       ...
       '9679459', '9639322', '9752896', '9430799', '9417621', '9754087',
       '9739284', '9751531', '4669875', '9751877'],
      dtype='object', length=4247)

In [68]:
content_recs = get_recommendations(article_id, cosine_similarity_df)

In [70]:
combined_recs = list(content_recs) + list(cf_recs)

In [81]:
# Combine content-based and collaborative filtering
def hybrid_recommendations(user_id, article_id, user_matrix_df, article_matrix_df, cosine_similarity_df):
    # Get content-based recommendations
    content_recs, specific_article_info = get_recommendations(article_id, cosine_similarity_df)

    # Get collaborative filtering recommendations
    #user_idx = interaction_matrix_df.index.get_loc(user_id)
    user_pref = user_matrix_df.loc[user_id, :]
    cf_scores = user_pref.dot(article_matrix_df.values.T)
    cf_recs = interaction_matrix_df.columns[cf_scores.argsort()[::-1]]

    # Combine recommendations
    combined_recs = list(content_recs) + list(cf_recs)
    return combined_recs[:10], specific_article_info

# Example usage (We pass User_id and article_id that has in view in order to see if we will click it)
combined_recommendations,specific_article_info =  hybrid_recommendations(750497, 9749224, user_matrix_df, article_matrix_df, cosine_similarity_df)

KeyError: 119053

In [80]:
history_val_df

,user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
0,750497,"[2023-05-04T09:42:39.000000, 2023-05-04T09:43:...","[100.0, 65.0, 100.0, 100.0, 100.0, 100.0, 100....","[9749224, 9749156, 9749224, 9748948, 9748980, ...","[49.0, 5.0, 7.0, 151.0, 214.0, 199.0, 22.0, 64..."
1,22779,"[2023-05-04T07:53:42.000000, 2023-05-04T15:59:...","[52.0, 39.0, 62.0, 38.0, 74.0, 19.0, 30.0, 56....","[9749025, 9750090, 9750015, 9750161, 9745750, ...","[4.0, 16.0, 2.0, 9.0, 40.0, 7.0, 9.0, 8.0, 18...."
2,373598,"[2023-05-04T07:51:58.000000, 2023-05-04T09:59:...","[nan, nan, nan, 59.0, 33.0, 75.0, nan, nan, 76...","[9514481, 9514481, 9111040, 9750389, 9750307, ...","[0.0, 0.0, 0.0, 3.0, 9.0, 117.0, 39.0, 0.0, 8...."
3,383378,"[2023-05-04T07:27:57.000000, 2023-05-04T07:29:...","[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100...","[9747490, 9749036, 9749025, 9748792, 9748592, ...","[85.0, 18.0, 133.0, 191.0, 331.0, 56.0, 43.0, ..."
4,411733,"[2023-05-04T17:09:09.000000, 2023-05-04T17:09:...","[20.0, 14.0, 61.0, 55.0, 21.0, 81.0, 100.0, 10...","[9750081, 9750111, 9750039, 9749948, 9749729, ...","[2.0, 4.0, 6.0, 9.0, 1.0, 30.0, 37.0, 5.0, 3.0..."
...,...,...,...,...,...
1557,626090,"[2023-05-16T15:24:43.000000, 2023-05-16T15:24:...","[92.0, 97.0, 90.0, 32.0, 24.0]","[9768387, 9766599, 9768152, 9767554, 9767342]","[9.0, 104.0, 15.0, 15.0, 64.0]"
1558,1143249,"[2023-05-24T13:08:55.000000, 2023-05-24T13:09:...","[83.0, 94.0, 100.0, 100.0, 100.0, 100.0, 100.0...","[9779227, 9779184, 9778813, 9778710, 9777164, ...","[14.0, 72.0, 33.0, 103.0, 40.0, 109.0, 272.0, ..."
1559,109970,"[2023-05-19T12:14:25.000000, 2023-05-19T12:15:...","[100.0, 62.0, 100.0, 100.0, 100.0, 100.0]","[9772517, 9772389, 9772517, 9772227, 9771113, ...","[40.0, 9.0, 14.0, 75.0, 26.0, 30.0]"
1560,330156,"[2023-05-25T04:30:51.000000, 2023-05-25T04:34:...","[100.0, 100.0, 100.0, 98.0, 37.0, 35.0, nan]","[9780096, 9777822, 9777822, 9779517, 9779511, ...","[153.0, 235.0, 10.0, 57.0, 289.0, 8.0, 0.0]"


In [79]:
specific_article_info

,article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,...,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
7,3001353,Natascha var ikke den første,"Politiet frygter nu, at Nataschas bortfører ha...",2023-06-29 06:20:33,False,Sagen om den østriske Natascha og hendes bortf...,2006-08-31 08:06:45,[3150850],article_default,https://ekstrabladet.dk/krimi/article3001353.ece,...,[],"[Kriminalitet, Personfarlig kriminalitet]",140,[],krimi,NaN,NaN,NaN,0.9955,Negative
